### Auto Bayesian Neural Networks


autoBNN on EUR/USD

In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import autobnn as ab
import jax
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

2025-06-23 23:55:05.762391: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750737305.784966   38712 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750737305.794793   38712 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
eurusd = yf.download("EURUSD=X", start="2005-01-01", end="2025-06-22")

/tmp/ipykernel_38712/3926671931.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  eurusd = yf.download("EURUSD=X", start="2005-01-01", end="2025-06-22")
[*********************100%***********************]  1 of 1 completed


In [3]:
print(eurusd.head())

Price          Close      High       Low      Open   Volume
Ticker      EURUSD=X  EURUSD=X  EURUSD=X  EURUSD=X EURUSD=X
Date                                                       
2005-01-03  1.347001  1.358105  1.340195  1.358105        0
2005-01-04  1.328198  1.349601  1.326102  1.347001        0
2005-01-05  1.328004  1.330300  1.322297  1.328304        0
2005-01-06  1.318305  1.327898  1.316396  1.327898        0
2005-01-07  1.306097  1.324802  1.302999  1.318200        0
